In [1]:
import os
import re
from dataclasses import dataclass
from typing import List
import shutil
import io, sys, types

from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

import torch.nn as nn
import torch
import torch.nn.functional as F

import pandas as pd
import torchvision
from PIL import Image
import torchvision.transforms as transforms
import numpy as np

## Make student directories

In [2]:
exclude = [
    "arceatom","ballspencer","bhargavaaman","carnaffanadam","castillejosyviel",
    "chengbenjamin","chenjacky","chiamkatarina","chudavid","cyphuskieran",
]


if False:

    student_names = pd.value_counts([x.split('_')[0] for x in os.listdir('./submissions/')]).keys()
    
    for student in student_names:
        if student in exclude:
            continue

        if not os.path.isdir('./student_submissions/{}'.format(student)):
            os.makedirs('./student_submissions/{}'.format(student))

        shutil.copyfile(
            './ECE324_A4P2_rubric.xlsx', 
            './student_submissions/{}/ECE324_A4P2_rubric.xlsx'.format(student)
        )

        student_files = [x for x in os.listdir('./submissions/') if x.startswith(student)]

        for student_file in student_files:
            shutil.copyfile('./submissions/{}'.format(student_file), './student_submissions/{}/{}'.format(
                student,
                student_file
            ))


### Construct data class

## Construct Test dataset

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor()]
)

train_dataset = torchvision.datasets.ImageFolder('../ece324_assignment_4_1/training_v2', transform=transform)
train_data = torch.utils.data.DataLoader(train_dataset)

training_stds = []
training_means = []

for image, label in train_data:
    training_means.append(image.mean(axis=(0, 2, 3)).numpy())
    training_stds.append(image.std(axis=(0, 2, 3)).numpy())
    
training_means = np.array(training_means).mean(axis=0)
training_stds = np.array(training_stds).mean(axis=0)

In [4]:
transform = transforms.Compose(
    [
        transforms.ToTensor(), 
        transforms.Normalize(training_means, training_stds)
    ]
)


test_dataset = torchvision.datasets.ImageFolder(
    '../ece324_assignment_4_1/data/test/',
    transform=transform
)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=10000)

In [5]:
@dataclass
class StudentModel:
    model_name: str
    student_name: str
    model_size: str
    model_perf: float
        
    
student_models = []
model_files = os.listdir('./student_models/')

for model_file in model_files:
    student_name = re.split('_|\.', model_file)[0]
    model_size = re.split('_|\.', model_file)[-2].lower()
    
    student_models.append(
        StudentModel(
            model_file,
            student_name,
            'small' if 'small' in model_size else 'big',
            -1)
    )
# Sort models by student name
student_models = sorted(
    student_models, key=lambda x: x.student_name
)
    
print(
    "Student model sizes: \n",
    pd.value_counts([x.model_size for x in student_models])
)
print('\n')
print(
    "Number of models per strudent: \n",
    pd.value_counts(pd.value_counts([x.student_name for x in student_models]).values)
)

Student model sizes: 
 big      51
small    50
dtype: int64


Number of models per strudent: 
 2    50
1     1
dtype: int64


In [6]:
student_models2 = []
model_files2 = os.listdir('./submissions/')
model_files2 = [x for x  in model_files2 if x.endswith('.pt')]

for model_file in model_files2:
    student_name = re.split('_|\.', model_file)[0]
    model_size = re.split('_|\.', model_file)[-2].lower()
    
    student_models2.append(
        StudentModel(
            model_file,
            student_name,
            'small' if 'small' in model_size else 'big',
            -1)
    )

student_models2 = sorted(
    student_models2, key=lambda x: x.student_name
)    
    
print(
    "Student model sizes: \n",
    pd.value_counts([x.model_size for x in student_models2])
)
print('\n')
print(
    "Number of models per strudent: \n",
    pd.value_counts(pd.value_counts([x.student_name for x in student_models2]).values)
)

Student model sizes: 
 small    52
big      51
dtype: int64


Number of models per strudent: 
 2    51
1     1
dtype: int64


In [7]:
# Submitted to the follow up submission but not the original
for model in student_models:
    if model.student_name not in [x.student_name for x in student_models2]:
        print(model)
        print('\n')

StudentModel(model_name='jianggeoffrey_206684_10241007_fullsmallestbestmodel.pt2', student_name='jianggeoffrey', model_size='small', model_perf=-1)


StudentModel(model_name='jianggeoffrey_206684_10241005_fullbestmodel.pt2', student_name='jianggeoffrey', model_size='big', model_perf=-1)


StudentModel(model_name='kadakiaarsh_212358_10190839_fullbestmodel.pt2', student_name='kadakiaarsh', model_size='big', model_perf=-1)


StudentModel(model_name='kadakiaarsh_212358_10190840_fullsmallestbestmodel.pt2', student_name='kadakiaarsh', model_size='small', model_perf=-1)


StudentModel(model_name='sahakesmat_181890_10235448_fullsmallestbestmodel.pt2', student_name='sahakesmat', model_size='small', model_perf=-1)


StudentModel(model_name='sahakesmat_181890_10235447_fullbestmodel.pt2', student_name='sahakesmat', model_size='big', model_perf=-1)




In [8]:
# Submitted to the original submission but not the follow up
for model in student_models2:
    if model.student_name not in [x.student_name for x in student_models]:
        print(model)
        print('\n')

StudentModel(model_name='malmirianmobin_LATE_203626_9820870_MyBest-1.pt', student_name='malmirianmobin', model_size='big', model_perf=-1)


StudentModel(model_name='malmirianmobin_LATE_203626_9820871_MyBestSmall-1.pt', student_name='malmirianmobin', model_size='small', model_perf=-1)


StudentModel(model_name='panwarshashwat_LATE_30849_9822129_MyBest.pt', student_name='panwarshashwat', model_size='big', model_perf=-1)


StudentModel(model_name='panwarshashwat_LATE_30849_9822131_MyBestSmall.pt', student_name='panwarshashwat', model_size='small', model_perf=-1)


StudentModel(model_name='sorbopeter_LATE_27311_9821418_MyBest.pt', student_name='sorbopeter', model_size='big', model_perf=-1)


StudentModel(model_name='sorbopeter_LATE_27311_9821419_MyBestSmall.pt', student_name='sorbopeter', model_size='small', model_perf=-1)


StudentModel(model_name='zhuangkathy_LATE_204758_9820518_MyBestSmall-2.pt', student_name='zhuangkathy', model_size='small', model_perf=-1)


StudentModel(model_name='z